In [2]:
!pip install -U --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 328.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 292.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 199.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 317.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 320.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 175.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 297.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 267.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exist

In [3]:
import json
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import bitsandbytes
from typing import List

2025-05-05 05:42:27.466903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746423747.715754      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746423747.778163      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def tokenize(example):
    input_text = example["event_text"]
    output_text = json.dumps(example["output"], ensure_ascii=False)
    full_text = f"{input_text}\n{output_text}"

    # Tokenize the full text
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=512,
    )

    # Re-tokenize input_text to find its length
    input_ids = tokenizer(input_text, truncation=True, max_length=512)["input_ids"]
    input_len = len(input_ids)

    # Create labels: ignore input part with -100
    labels = tokenized["input_ids"][:]
    labels[:input_len] = [-100] * input_len
    tokenized["labels"] = labels

    return tokenized


In [5]:
# Load and prepare data
data_path = "/kaggle/input/keyword-extraction-calender-dataset/event_text_mapping.jsonl"
samples = []
with open(data_path, "r") as f:
    for line in f:
        obj = json.loads(line)
        samples.append(obj)

dataset = Dataset.from_list(samples)
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-360M")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize
tokenized_dataset = dataset.map(tokenize, batched=False)

# Save
save_path = "./tokenized_event_dataset"
tokenized_dataset.save_to_disk(save_path)


tokenizer_config.json:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

Map:   0%|          | 0/712 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/712 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/80 [00:00<?, ? examples/s]

In [7]:
# 1. Load Tokenized Dataset
from datasets import load_from_disk

tokenized_dataset = load_from_disk("./tokenized_event_dataset")
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

# 2. Load Model (full precision)
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "HuggingFaceTB/SmolLM-360M"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. Prepare model with LoRA
from peft import LoraConfig, get_peft_model

target_modules = [
    "q_proj", "v_proj", "k_proj", "o_proj", "up_proj", "down_proj"
]

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

# 4. Print trainable parameter stats
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params} || All params: {all_params} || Trainable%: {100 * trainable_params / all_params:.2f}")

print_trainable_parameters(model)



config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Trainable params: 6881280 || All params: 368702400 || Trainable%: 1.87


In [8]:
# 5. Define TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results_lora",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs_lora",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=2e-5,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,
    disable_tqdm=False,
    report_to="none",
    fp16=True  # Optional: Set to True if using FP16
)

# 6. Define Trainer and Train
from transformers import Trainer, DataCollatorForLanguageModeling

# Decoder-only LM needs a proper collator that preserves labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 7. Train and Save
trainer.train()
trainer.save_model("./fine_tuned_lora_model")
print("✅ LoRA fine-tuning complete! Model saved to ./fine_tuned_lora_model")


/tmp/ipykernel_31/2852160393.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.531700,1.456918
2,0.946600,0.941099
3,0.719400,0.729572
4,0.645600,0.653334
5,0.620100,0.614431
6,0.562600,0.592794
7,0.571400,0.581307
8,0.560400,0.575478
9,0.550200,0.573506
10,0.523800,0.573182


✅ LoRA fine-tuning complete! Model saved to ./fine_tuned_lora_model


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch

# Load tokenizer
model_id = "HuggingFaceTB/SmolLM-360M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load base model + LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, "./fine_tuned_lora_model")
model.eval()



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🔍 Extracted structured event:
 
{"action": "Hiking", "attendees": ["Leo"], "date": "06/04/2024", "duration": "2 hours", "location": "Hillside Trail", "notes": null, "recurrence": null, "time": "6:00 AM"}
{"action": "Hiking", "attendees": ["Leo"], "date": "06/04/2024", "duration": null, "location": "Hillside Trail", "notes": null, "recurrence": null,


In [15]:

def inference(input_text):    
    # Prepare input
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(model.device)
    
    # Generate
    with torch.no_grad():
        generated_ids = model.generate(
            **input_ids,
            max_new_tokens=128,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode output
    output = tokenizer.decode(generated_ids[0][input_ids['input_ids'].shape[1]:], skip_special_tokens=True)
    print("🔍 Extracted structured event:\n", output)

In [16]:
inference(input_text = "Meeting on 05 - December - 2023, 3pm, lasting 1 hour, with Sarah and James on Google Meet.")

🔍 Extracted structured event:
 
{"action": "Meeting", "attendees": ["Sarah", "James"], "date": "05/12/2023", "duration": "1 hour", "location": "Google Meet", "notes": null, "recurrence": null, "time": "3:00 PM"}
{"action": "Meeting", "attendees": ["Sarah", "James"], "date": "05/12/2023", "duration": null, "location": "Google Meet", "notes": null, "recurrence": null,


In [22]:
inference("Arrange a team-building activity with Ella, Mason, and Harper on 12th, November 2025 at 10 AM in the community park for 3 hours.")

🔍 Extracted structured event:
 
{"action": "Arrange a team-building activity", "attendees": ["Ella", "Mason", "Harper"], "date": "12/11/2025", "duration": "3 hours", "location": "community park", "notes": null, "recurrence": null, "time": "10:00 AM"}
{"action": "Arrange a team-building activity", "attendees": ["Ella", "Mason", "Harper"], "date": "12/11/2025", "


In [9]:
import shutil
from IPython.display import FileLink

def download_folder_as_zip(folder_path = "/kaggle/working/fine_tuned_lora_model", zip_name="finetuned_way2.zip"):
    """
    Zips the given folder and creates a download link for it in a Kaggle notebook.
    
    Args:
        folder_path (str): Path to the folder to be zipped.
        zip_name (str): Name of the zip file to create.
    """
    # Zip the folder
    shutil.make_archive(zip_name.replace(".zip", ""), 'zip', folder_path)
    
    # Provide a download link
    display(FileLink(zip_name))

In [10]:
download_folder_as_zip()

/kaggle/working/finetuned_way2.zip